1. **Reto 1: Sistema de Reservas de Hotel**
   - Implementa un sistema de reservas para un hotel donde existan diferentes tipos de habitaciones: estándar, deluxe y suite. Crea una clase base `Habitacion` con atributos comunes y clases derivadas para cada tipo de habitación que añadan características específicas (como servicios adicionales). Asegúrate de sobrescribir métodos en las subclases cuando sea necesario.
   
### Reto 3: Sistema de Reservas en un Hotel

#### Descripción:
Diseña un sistema para manejar reservas en un hotel. Las reservas pueden ser de diferentes tipos: `ReservaHabitacion`, `ReservaEvento`, y `ReservaPaquete`. Cada tipo de reserva tiene un método `calcular_costo()` que calcula el costo de la reserva basado en diferentes criterios.

- **ReservaHabitacion**: Calcula el costo basado en la cantidad de noches y el tipo de habitación.
- **ReservaEvento**: Calcula el costo en función del tamaño del evento y los servicios adicionales.
- **ReservaPaquete**: Calcula el costo combinando varios servicios (habitación, comida, spa, etc.).

#### Requerimientos:
1. Define una clase abstracta `Reserva` con un método abstracto `calcular_costo()`.
2. Implementa las subclases `ReservaHabitacion`, `ReservaEvento`, y `ReservaPaquete` con su propia versión del método `calcular_costo()`.
3. Crea un sistema que maneje una lista de reservas y calcule el costo total para todas las reservas en el hotel.


In [25]:
from dataclasses import dataclass, field
from typing import List, Optional
from datetime import datetime, timedelta
import random

@dataclass
class Empleado:
    nombre: str
    estado: bool = True  # True si está disponible, False si no

    def asignar_tarea(self) -> str:
        if self.estado:
            self.estado = False
            return f"Trabajador {self.nombre} asignado."
        return f"El trabajador {self.nombre} no está disponible."

    def liberar(self):
        self.estado = True

@dataclass
class Habitacion:
    id: str
    precio: int  # El precio se definirá en las clases hijas
    disponibilidad: bool = True  # False si está ocupada
    reservas: List['Reserva'] = field(default_factory=list)
    fechas_ocupadas: List[datetime] = field(default_factory=list)

    def verificar_disponibilidad(self, fecha_entrada: datetime, fecha_salida: datetime) -> bool:
        """Verifica si la habitación está disponible en el rango de fechas dado."""
        rango_fechas = [fecha_entrada + timedelta(days=i) for i in range((fecha_salida - fecha_entrada).days + 1)]
        for fecha in rango_fechas:
            if fecha in self.fechas_ocupadas:
                return False
        return True

    def agregar_reserva(self, reserva: 'Reserva'):
        self.reservas.append(reserva)
        self.fechas_ocupadas += reserva.calcular_rango_fechas()
        self.disponibilidad = False

    def liberar(self):
        self.disponibilidad = True

@dataclass
class Hotel:
    empleados: List[Empleado]
    habitaciones: List[Habitacion]

    def trabajador_disponible(self) -> Optional[Empleado]:
        """Retorna un trabajador disponible al azar, o None si no hay ninguno."""
        empleados_disponibles = [empleado for empleado in self.empleados if empleado.estado]
        if empleados_disponibles:
            return random.choice(empleados_disponibles)
        return None

    def asignar_trabajador(self) -> str:
        """Asigna un trabajador disponible para una tarea."""
        trabajador = self.trabajador_disponible()
        if trabajador:
            return trabajador.asignar_tarea()
        return "No hay trabajadores disponibles en este momento."

@dataclass
class Suite(Habitacion):
    precio: int = 60000

    def servicio_al_cuarto(self, hotel: Hotel) -> str:
        """Solicita servicio al cuarto y asigna un trabajador."""
        resultado = hotel.asignar_trabajador()
        return f"Solicitando servicio al cuarto...\n{resultado}"

@dataclass
class Estandar(Habitacion):
    precio: int = 30000

@dataclass
class Reserva:
    fecha_entrada: datetime
    fecha_salida: datetime
    habitacion: Habitacion

    def calcular_rango_fechas(self) -> List[datetime]:
        """Calcula el rango de fechas de la reserva."""
        return [self.fecha_entrada + timedelta(days=i) for i in range((self.fecha_salida - self.fecha_entrada).days + 1)]

    def reservar(self) -> str:
        """Realiza una reserva si la habitación está disponible."""
        hoy = datetime.today()
        if self.fecha_entrada < hoy:
            return "No se puede reservar una habitación para una fecha anterior a hoy."
        
        if self.fecha_salida <= self.fecha_entrada:
            return "La fecha de salida debe ser mayor a la fecha de entrada."
        
        if self.habitacion.verificar_disponibilidad(self.fecha_entrada, self.fecha_salida):
            self.habitacion.agregar_reserva(self)
            return "Reserva exitosa."
        return "No se puede reservar, la habitación no está disponible en esas fechas."

    def cancelar_reserva(self) -> str:
        """Cancela una reserva y libera la habitación."""
        if self in self.habitacion.reservas:
            self.habitacion.reservas.remove(self)
            self.habitacion.fechas_ocupadas = [fecha for fecha in self.habitacion.fechas_ocupadas 
                                               if fecha not in self.calcular_rango_fechas()]
            if not self.habitacion.reservas:  # Si no hay más reservas
                self.habitacion.liberar()
            return "Reserva cancelada y habitación liberada."
        return "La reserva no existe."

# Uso del sistema

empleado1 = Empleado(nombre="Carlos")
empleado2 = Empleado(nombre="Laura")
empleado3 = Empleado(nombre="María")

habitacion_estandar = Estandar(id="E1")
habitacion_suite = Suite(id="S1")
habitacion_suite2 = Suite(id="S2")

hotel = Hotel(empleados=[empleado1, empleado2, empleado3], habitaciones=[habitacion_estandar, habitacion_suite, habitacion_suite2])

# Haciendo una reserva
reserva1 = Reserva(fecha_entrada=datetime(2024, 2, 2), fecha_salida=datetime(2024, 2, 3), habitacion=habitacion_estandar)
print(reserva1.reservar())  # Reserva la habitación si está disponible

# Solicitando servicio al cuarto en una suite
print(habitacion_suite.servicio_al_cuarto(hotel))

# Cancelar reserva
print(reserva1.cancelar_reserva())


No se puede reservar una habitación para una fecha anterior a hoy.
Solicitando servicio al cuarto...
Trabajador Carlos asignado.
La reserva no existe.


---
# Retos clases abstractas
4 . Retos (0,1)

### Ejercicio 1:
Define una clase abstracta `InstrumentoMusical` con dos métodos abstractos: `tocar` y `afinar`. Luego, crea dos clases concretas que hereden de esta clase: `Guitarra` y `Piano`, implementando los métodos abstractos en ambas clases.

### Ejercicio 2:
Crea una clase abstracta `Vehiculo` con un método abstracto `arrancar` y un método concreto `detener`. Define dos clases `Coche` y `Moto` que hereden de `Vehiculo` e implementen su propio método `arrancar`.

### Ejercicio 3:
Imagina que estás diseñando un sistema de pagos. Crea una clase abstracta `MetodoDePago` con un método abstracto `procesar_pago`. Luego, crea dos clases concretas `TarjetaCredito` y `PayPal` que implementen el método `procesar_pago` de manera diferente.

---

In [23]:
from abc import ABC, abstractmethod

class InstrumentoMusical(ABC):
    @abstractmethod
    def tocar(self):
        pass

    @abstractmethod
    def afinar(self):
        pass

class Guitarra(InstrumentoMusical):
    def tocar(self):
        return "Tocando la guitarra"
    
    def afinar(self):
        return "Afinando la guitarra"

class Piano(InstrumentoMusical):
    def tocar(self):
        return "Tocando el piano "
    
    def afinar(self):
        return "Afinando el piano"


guitarra = Guitarra()
piano = Piano()

print(guitarra.tocar()) 
print(guitarra.afinar())
print(piano.tocar())    
print(piano.afinar())


Tocando la guitarra
Afinando la guitarra
Tocando el piano 
Afinando el piano


In [22]:
from abc import ABC, abstractmethod


class Vehiculo(ABC):
    @abstractmethod
    def arrancar(self):
        pass
    
    def detener(self):
        return "El vehículo se ha detenido."

class Carro(Vehiculo):
    def arrancar(self):
        return "El carro ha arrancado "


class Moto(Vehiculo):
    def arrancar(self):
        return "La moto ha arrancado"


carro = Carro()
moto = Moto()

print(carro.arrancar()) 
print(carro.detener())   
print(moto.arrancar())  
print(moto.detener())

El carro ha arrancado 
El vehículo se ha detenido.
La moto ha arrancado
El vehículo se ha detenido.


In [26]:
from abc import ABC, abstractmethod

# Clase abstracta
class MetodoDePago(ABC):
    @abstractmethod
    def procesar_pago(self, cantidad):
        pass

# Clase concreta TarjetaCredito
class TarjetaCredito(MetodoDePago):
    def procesar_pago(self, cantidad):
        return f"Procesando un pago de {cantidad} con tarjeta de crédito 💳"

# Clase concreta PayPal
class PayPal(MetodoDePago):
    def procesar_pago(self, cantidad):
        return f"Procesando un pago de {cantidad} con PayPal 🧾"

# Ejemplo de uso
tarjeta = TarjetaCredito()
paypal = PayPal()

print(tarjeta.procesar_pago(100))  # "Procesando un pago de 100 con tarjeta de crédito"
print(paypal.procesar_pago(200))   # "Procesando un pago de 200 con PayPal"


Procesando un pago de 100 con tarjeta de crédito 💳
Procesando un pago de 200 con PayPal 🧾
